In [ ]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import random

In [ ]:
Data = np.zeros([52, 8])


file_dir = './NEW DATA/DOE' # csv위치 폴더
file_list = os.listdir(file_dir)

for i in range(len(file_list)):
    tmp = pd.read_csv(file_dir + '/' + file_list[i])                           # 파일 불러오기
    Data[i, :4]   = tmp.iloc[0, 1:]                                            # 초기값(모니터링 변수) 저장
    Data[i, 4:-2] = [float(file_list[i][3:6]), float(file_list[i][8:10])]      # P, I값 저장
    Data[i, -2] = tmp.shape[0]                                                 # 전체 공정시간 저장 (아마 사용 X)
    
    idx = np.where(tmp.iloc[:, 0] == 'eSTERILIZATION')[0][0]                   # 멸균 직전 시간까지 index 가져오기
    Data[i, -1] = tmp.iloc[:idx, :].shape[0]                                   # 조정단계까지 시간 저장

file_dir = './NEW DATA/OTHER' # csv위치 폴더
file_list = os.listdir(file_dir)
    
for i in range(len(file_list)):
    tmp = pd.read_csv(file_dir + '/' + file_list[i])                           # 파일 불러오기
    Data[i+45, :4]   = tmp.iloc[0, 1:]                                         # 초기값(모니터링 변수) 저장
    Data[i+45, 4:-2] = [float(file_list[i][3:6]), float(file_list[i][8:10])]   # P, I값 저장
    Data[i+45, -2] = tmp.shape[0]                                              # 전체 공정시간 저장 (아마 사용 X)
    
    idx = np.where(tmp.iloc[:, 0] == 'eSTERILIZATION')[0][0]                   # 멸균 직전 시간까지 index 가져오기
    Data[i+45, -1] = tmp.iloc[:idx, :].shape[0]                                # 조정단계까지 시간 저장
    
Data = pd.DataFrame(Data, columns = ['Plate Temp', 'Steam Temp', 'Vessel Pressure', 'Vessel Temp', 'P', 'I', 'Total Time', 'ADJ Time'])
Data

In [ ]:
Final_Acc = pd.read_csv('./Result/Final_Acc.csv')
Final_Acc

In [ ]:
x1 = np.arange(0.1, 4.1, 0.1)
x2 = np.arange(10, 91, 1)

PI_value = np.meshgrid(x1,x2)

In [ ]:
PI_value = np.zeros([x1.shape[0]*x2.shape[0], 2])
for i in range(x1.shape[0]):
    for j in range(x2.shape[0]):
        PI_value[i*x2.shape[0] + j, 0] = x1[i]
        PI_value[i*x2.shape[0] + j, 1] = x2[j]

In [ ]:
PI_value.shape

In [ ]:
idx = Final_Acc.sort_values(by='V_MAPE').iloc[0, 0]
LoadModel = keras.models.load_model('./Model/Model_%d.h5'%(idx))


for i in range(Data.shape[0]):
    temp = Data.iloc[i,:4]
    Total_value = np.zeros([PI_value.shape[0], 6])
    for j in range(PI_value.shape[0]):
        Total_value[j, :4] = temp
        Total_value[j, 4:] = PI_value[j, :]
        
    random_thickness = pd.DataFrame(LoadModel.predict(Total_value, verbose=0))
    random_thickness_rank = random_thickness.sort_values([0],ascending = True)

    print(i+1, ':', random_thickness_rank.iloc[0], Total_value[random_thickness_rank.index[0],4:])

In [ ]:
pd.concat([ Data.iloc[:,4:6], Data.iloc[:,2:4], Data.iloc[:,:2], Data.iloc[:,-1]], axis = 1)

In [ ]:
Final_Acc.sort_values(by='V_MAPE')

In [ ]:
np.mean(Data.iloc[:,-1])